In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
ls

Train Transformer 1.ipynb  models.py
dataloader.py


# First, prepare the data

In [3]:
from dataloader import *

In [4]:
# load every dataset manually and create torch objects
comment_df = pd.read_csv("../data/attack_annotated_comments.tsv", sep ='\t')
body_df = pd.read_csv("../data/fake_news_bodies.csv")
stance_df = pd.read_csv("../data/fake_news_stances.csv")
vocab = Vocabulary([comment_df["comment"], body_df["articleBody"], stance_df["Headline"]])
annotation_df = pd.read_csv("../data/attack_annotations.tsv",  sep='\t')

wiki_dataset = WikiDataset(comment_df, annotation_df, vocab)
fake_news_dataset = FakeNewsDataset(body_df, stance_df, vocab)

In [5]:
comment_df.head(2)

,rev_id,comment,year,logged_in,ns,sample,split
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train


In [6]:
body_df.head(2)

,Body ID,articleBody,sentence_as_idx
0,0,A small meteorite crashed into a wooded area i...,"[2, 126, 2627, 7088, 36713, 1541, 126, 64723, ..."
1,4,Last week we hinted at what was to come as Ebo...,"[2, 978, 9959, 301, 27551, 181, 184, 243, 21, ..."


In [7]:
stance_df.head(2)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree


In [8]:
# check label correspondence
stance_df['Stance'].unique()

array(['unrelated', 'agree', 'disagree', 'discuss'], dtype=object)

In [9]:
annotation_df.head(2)

,rev_id,worker_id,quoting_attack,recipient_attack,third_party_attack,other_attack,attack
0,37675,1362,0.0,0.0,0.0,0.0,0.0
1,37675,2408,0.0,0.0,0.0,0.0,0.0


In [10]:
# check label bias
fake_news_dataset.y.mean(axis=0)

array([0.73130953, 0.07360122, 0.01680941, 0.17827984])

In [11]:
# check label bias
wiki_dataset.y.mean(axis=0)

array([0.88270731, 0.11729269])

# Once the data is loaded, prepare the model

In [18]:
# simple transformer model, cant be used for classification
vocab_size = len(vocab)
labels = len(vocab[0][1])
embedding_dim = 64
nhead = 1
hidden_dim = 32
num_layers = 1
feedforward_dim = 64

In [19]:
from models import *

In [20]:
model = TransformerClassifier(vocab_size, labels, embedding_dim, nhead, feedforward_dim, num_layers)

In [24]:
model(torch.rand(1, 12, vocab_size).long())

RuntimeError: The size of tensor a (474033) must match the size of tensor b (12) at non-singleton dimension 2

In [14]:
def train_model(encoder, dataset, batch_size=32):
    print("training encoder")
    n = len(dataset)
    batch_order = np.arange(n)
    np.random.shuffle(batch_order)
    for i in range(0, n, batch_size):
        curr_batch = dataset[batch_order[i:min(i+batch_size, n)]]

In [15]:
train_model(encoder, wiki_dataset)

NameError: name 'encoder' is not defined